In [1]:
from platform import python_version
print(python_version())

3.8.13


In [2]:
import ydata_synthetic
import pandas as pd
from ydata_synthetic.synthesizers import ModelParameters
from ydata_synthetic.synthesizers.timeseries import TimeGAN

import os 
import numpy as np


In [3]:
pathON="C:/Users/pierr/Documents_kanop/Shps/Pierre_points_for_louis/Bureau/interpo_ONOFF/ON_data/"


In [4]:
#Storing in a list the adresses of all the data files in the notebook

data_files_ON=[]
file_names_ON=[]

data_files_ALL=[]
data_files_OFF=[]
file_names_OFF=[]
file_names_ALL=[]
data_files_OFF=[]
#Storing in a list the adresses of all the data files in the notebook

for filename in os.listdir(pathON):
    f = os.path.join(pathON,filename)
    if os.path.isfile(f):
        data_files_ON.append(f)
        file_names_ON.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)

In [5]:
#Using data_files with only the 36 binning
Filter=True
binning=37
nbfeatures=1



if Filter==True:
    
    idx_OFF=[]
    idx_ON=[]
    dataON=[]
    dataOFF=[]
    for i in range(len(data_files_OFF)):
        dataframe = pd.read_csv(data_files_OFF[i])
        lg = len(dataframe)
        
        if lg==binning:
            idx_OFF.append(i)
    
    for i in range(len(data_files_ON)):
        dataframe = pd.read_csv(data_files_ON[i])
        lg = len(dataframe)
#         print(lg)
        if lg==binning:
            idx_ON.append(i)

    #Construct csv dataframe with features of LC and LC dir
    # RA, DEC ,class, file name 

    #Future input shape : 3511 sources, 2 features (as time series) , x binnings
    for i in range(len(idx_OFF)):

        a=idx_OFF[i]
        dataOFF.append(data_files_OFF[a])
    for i in range(len(idx_ON)):

        a=idx_ON[i]
        dataON.append(data_files_ON[a])

idx = idx_OFF+idx_ON


In [6]:


seq_len = 37      # Timesteps
n_seq = 1         # Features
hidden_dim = 37    # Hidden units for generator (GRU & LSTM).                 # Also decides output_units for generator
gamma = 1           # Used for discriminator loss
noise_dim = 36 #37 ?      # Used by generator as a starter dimension
dim = 128   
batch_size = 11
learning_rate = 3e-2
gan_args = ModelParameters(batch_size=batch_size,
                           lr=learning_rate,
                           noise_dim=noise_dim,
                           layers_dim=dim)

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

lgON=len(dataON)
a=binning
b=nbfeatures
# c=lgON
c=lgON
# multivariate
y=np.zeros((2,lg))
bigdata= np.zeros((c,a,b))


min_max_scaler = preprocessing.MinMaxScaler()

for j in range(len(dataON)):
    dataframe=pd.read_csv(dataON[j])
    dataframe.columns=['Iteration','MJD','Flux','Delta Flux','Photon Index','Delta Index','TS','fratio']
    Flux = dataframe['Flux'].values.reshape(-1, 1)
#     Photon_idx = dataframe['Photon Index']
    Flux= min_max_scaler.fit_transform(Flux)    
    bigdata[j]=Flux
#         bigdata[j][kk][1]=Photon_idx[kk]

        
# bigdata=bigdata.reshape(c,binning,nbfeatures)    

synth = TimeGAN(model_parameters=gan_args, hidden_dim=hidden_dim, seq_len=binning, n_seq=1, gamma=1)
synth.train(bigdata, train_steps=300)
    

# synth_data = synth.sample(binning)


Joint networks training:  99%|█████████▊| 296/300 [06:05<00:04,  1.01s/it]

In [ ]:
print(Flux.shape)

In [ ]:
synth_data = synth.sample(3)

print(synth_data[0])

In [ ]:
# pathON="C:/Users/pierr/Documents_kanop/Shps/Pierre_points_for_louis/Bureau/interpo_ONOFF/ON_synthetic/"

# for i in range(306):
    
#     a = synth_data[i]
#     df = pd.DataFrame(a)
#     new_file_path = pathON+'ON_synthetic_'+str(i)+'.csv'
#     df.to_csv(new_file_path)
    

In [ ]:
import matplotlib.pyplot as plt
plt.plot(dataframe["MJD"],synth_data[2])

In [ ]:
plt.scatter(dataframe["MJD"],Flux)

In [ ]:
print(exa)